<a href="https://colab.research.google.com/github/bonsoul/IMDB-Movie-Reviews---Sentiment-Analysis---LSTM/blob/main/IMDB_Movie_Reviews_Sentiment_Analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

## Importing the Dependencies

In [9]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [4]:
kaggle_dictionary = json.load(open("/content/kaggle.json"))


In [5]:
# set up Kaggle API

os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [3]:
#!/bin/bash
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 51% 13.0M/25.7M [00:00<00:00, 56.9MB/s]
100% 25.7M/25.7M [00:00<00:00, 81.1MB/s]


In [4]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  sample_data


In [11]:
# unzip the dataset file

with ZipFile("/content/imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [12]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   sample_data


#Loading the Dataset

In [13]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [14]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [15]:
data.shape

(50000, 2)

In [17]:
data.replace({"sentiment": {"positive":1,"negative":0}}, inplace=True)

<ipython-input-17-1065e0669ca3>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive":1,"negative":0}}, inplace=True)


In [18]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [19]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [20]:
# split data into training data and test data

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [21]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


# **Preprocessing**

In [37]:
# tokenize  text data

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [25]:
print(X_train)

[[   1 1200    4 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [26]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  59   26  482 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [27]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [28]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


Building the LSTM(Long Short Term Memory)

In [41]:
model = Sequential()
model.add(Input(shape=(200,)))
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [42]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#traing the model

In [44]:
model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 80s 291ms/step - accuracy: 0.7036 - loss: 0.5530 - val_accuracy: 0.8491 - val_loss: 0.3503
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 285ms/step - accuracy: 0.8613 - loss: 0.3385 - val_accuracy: 0.8470 - val_loss: 0.3490
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 77s 264ms/step - accuracy: 0.8762 - loss: 0.3087 - val_accuracy: 0.8129 - val_loss: 0.4232
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 90s 295ms/step - accuracy: 0.8774 - loss: 0.3050 - val_accuracy: 0.8585 - val_loss: 0.3388
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 76s 303ms/step - accuracy: 0.9020 - loss: 0.2523 - val_accuracy: 0.8558 - val_loss: 0.3422
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 74s 273ms/step - accuracy: 0.9062 - loss: 0.2377 - val_accuracy: 0.8556 - val_loss: 0.3720
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 77s 252ms/step - accuracy: 0.9064 - loss: 0.2331 - val_accuracy: 0.8559 - val_loss: 0.3501
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 81s 247ms/step - accuracy: 0.9206 - loss: 0

**Model Evaluation**

In [45]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 31s 98ms/step - accuracy: 0.8716 - loss: 0.3689
Test Loss: 0.3637451231479645
Test Accuracy: 0.8762000203132629


**Building a Predictive System**

In [52]:
def predict_sentiment(text):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([text])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)[0][0]
  sentiment = "positive" if prediction > 0.5 else "negative"
  return sentiment

In [53]:
# example usage

new_review = "This movie was amazing! I loved every minute of it"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment} ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
The sentiment of the review is: positive 


In [54]:
# example usage

new_review = "This movie was terrible! I hated every minute of it"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
The sentiment of the review is: negative
